In [1]:
import datetime
import os
from pathlib import Path

import numpy as np
import pandas as pd

# This must be done BEFORE importing any library that uses tqdm
import tqdm
import xarray as xr
from hwt_mpas import MemoryDataSource, SubsetNetCDF4Backend
from tqdm import notebook

tqdm.tqdm = notebook.tqdm

from earth2studio.data import GEFS_FX, GFS, LandSeaMask, SurfaceGeoPotential
from earth2studio.io import NetCDF4Backend
from earth2studio.models.px import GraphCastOperational, GraphCastSmall
from earth2studio.run import deterministic

SCRATCH = Path(os.getenv("SCRATCH"))

In [2]:
def run(init_time, model, z, lsm, members=["gec00"] + [f"gep{p:02d}" for p in range(1, 31)]):
    forecast_length = 240
    forecast_step_hours = 6
    nsteps = forecast_length // forecast_step_hours

    output_dir = f"/glade/derecho/scratch/ahijevyc/ai-models/output/graphcast/{init_time:%Y%m%d%H}"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Ensemble forecast outputs will be saved in: {output_dir}")

    model_variables = model.input_coords()["variable"]
    vars_to_zero_fill = [v for v in model_variables if v.startswith("w") or v == "tp06"]

    vars_to_fetch = [v for v in model_variables if v not in vars_to_zero_fill]
    vars_to_fetch.remove("z")
    vars_to_fetch.remove("lsm")

    for member in members:
        output_filepath = os.path.join(output_dir, f"{member}.nc")
        lat_slice = slice(20, 60)
        lon_slice = slice(220, 300)

        if os.path.exists(output_filepath):
            try:
                with xr.open_dataset(output_filepath) as ds:
                    if len(ds.data_vars) != 85:
                        raise ValueError(
                            f"Incorrect number of data variables. Expected 85, found {len(ds.data_vars)}."
                        )
                    for dim_name, dim_size in ds.dims.items():
                        if dim_size == 0:
                            raise ValueError(f"Dimension '{dim_name}' has size 0.")
                    if any(ds.z500.squeeze().max(dim=["lat", "lon"]) > 1e30):
                        raise ValueError(f"bad data in {output_filepath}")
                    print(
                        f"Valid and complete forecast file already exists for member '{member}', skipping."
                    )
                    continue
            except Exception as e:
                print(
                    f"Found invalid or incomplete file for member '{member}', removing. Error: {e}"
                )
                os.remove(output_filepath)

        print(f"Fetching initial conditions for {member} at {init_time.isoformat()}...")
        gefs_source = GEFS_FX(member=member)
        initial_state_partial = gefs_source(init_time, [datetime.timedelta(hours=0)], vars_to_fetch)

        print(f"Regridding initial state for {member}...")
        lat = model.input_coords()["lat"]
        lon = model.input_coords()["lon"]
        wrapped = initial_state_partial.sel(lon=0).assign_coords(lon=360)
        initial_state_periodic = xr.concat([initial_state_partial, wrapped], dim="lon")
        initial_state_partial = initial_state_periodic.interp(lat=lat, lon=lon, method="linear")

        data_arrays_to_concat = [initial_state_partial]
        for var_name in vars_to_zero_fill:
            zero_array = xr.zeros_like(initial_state_partial.isel(variable=0))
            zero_array["variable"] = var_name
            data_arrays_to_concat.append(zero_array)
        data_arrays_to_concat.extend([z, lsm])
        initial_state = xr.concat(data_arrays_to_concat, dim="variable", coords="minimal")

        assert initial_state.notnull().all()
        initial_state = initial_state.sel(variable=model_variables).squeeze(
            dim="lead_time", drop=True
        )

        in_memory_source = MemoryDataSource(initial_state)

        print(f"Running forecast and subsetting for member '{member}'...")

        subset_writer = SubsetNetCDF4Backend(
            file_name=output_filepath,
            lat_slice=lat_slice,
            lon_slice=lon_slice,
            backend_kwargs={"mode": "w"},
        )

        deterministic([init_time], nsteps, model, in_memory_source, subset_writer)

        subset_writer.close()

        print(f"Successfully created forecast file: {output_filepath}")
        print(f"--- Finished forecast for member '{member}' ---")

    print(
        f"\n✅ All ensemble member forecasts for {init_time.date()} have been successfully generated."
    )

In [3]:
# --- Main Execution Block ---
model_class = GraphCastOperational
print(f"Initializing {model_class} model...")
model = model_class.load_model(model_class.load_default_package())
print("Model initialized successfully.")

model.input_coords()

Initializing <class 'earth2studio.models.px.graphcast_operational.GraphCastOperational'> model...


INFO:2025-11-13 13:12:10,764:jax._src.xla_bridge:925: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2025-11-13 13:12:10,764 - INFO - Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:2025-11-13 13:12:10,770:jax._src.xla_bridge:925: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
2025-11-13 13:12:10,770 - INFO - Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


Model initialized successfully.


OrderedDict([('batch', array([], dtype=float64)),
             ('time', array([], dtype=float64)),
             ('lead_time', array([-6,  0], dtype='timedelta64[h]')),
             ('variable',
              array(['t2m', 'msl', 'u10m', 'v10m', 'tp06', 't50', 't100', 't150',
                     't200', 't250', 't300', 't400', 't500', 't600', 't700', 't850',
                     't925', 't1000', 'z50', 'z100', 'z150', 'z200', 'z250', 'z300',
                     'z400', 'z500', 'z600', 'z700', 'z850', 'z925', 'z1000', 'u50',
                     'u100', 'u150', 'u200', 'u250', 'u300', 'u400', 'u500', 'u600',
                     'u700', 'u850', 'u925', 'u1000', 'v50', 'v100', 'v150', 'v200',
                     'v250', 'v300', 'v400', 'v500', 'v600', 'v700', 'v850', 'v925',
                     'v1000', 'w50', 'w100', 'w150', 'w200', 'w250', 'w300', 'w400',
                     'w500', 'w600', 'w700', 'w850', 'w925', 'w1000', 'q50', 'q100',
                     'q150', 'q200', 'q250',

In [4]:
# Load static data (z, lsm), fetching and saving as local files if not present.
print("Loading static data (z, lsm)...")
lat = model.input_coords()["lat"]
lon = model.input_coords()["lon"]

# Handle Geopotential (z)
print("Fetching z")
z = (
    # Tried cache=True but AttributeError: type object 'WholeFileCacheFileSystem' has no attribute '_cat_file'
    SurfaceGeoPotential(cache=False)([None])
    .sel(lat=lat, lon=lon)
    .squeeze()
)

# Handle Land-Sea Mask (lsm)
print("Fetching lsm")
lsm = LandSeaMask(cache=False)([None]).sel(lat=lat, lon=lon).squeeze()

print("Static data loaded successfully.")

Loading static data (z, lsm)...
Fetching z
Fetching lsm
Static data loaded successfully.


In [ ]:
for init_time in pd.to_datetime(pd.date_range("20230424", "20230531")):
    print(f"\n{'='*20} Starting Run for {init_time.date()} {'='*20}")
    run(init_time, model, z, lsm)

In [5]:
class GFSWithSfcFields(GFS):
    """
    Intercepts requests for specified variables and provides predefined data arrays.
    For all other variables, it falls back to the standard GFS implementation.
    """

    def __init__(self, custom_arrays: dict[str, xr.DataArray], *args, **kwargs):
        """
        Initializes with a dictionary of custom xarray.DataArrays,
        where the keys are the variable names.
        """
        super().__init__(*args, **kwargs)
        self.custom_arrays = custom_arrays
        self.custom_vars = list(custom_arrays.keys())

    def __call__(
        self,
        time: np.ndarray,
        variable: np.ndarray,
    ) -> xr.DataArray:
        # Identify which custom variables are in the current request
        requested_custom_vars = [v for v in self.custom_vars if v in variable]

        # Identify which variables need to be fetched from the standard GFS source
        other_vars = [v for v in variable if v not in self.custom_vars]

        # Fetch data for non-custom variables from the parent GFS class
        if other_vars:
            gfs_data = super().__call__(time, np.array(other_vars))
        else:
            gfs_data = None

        # Prepare a list of all data arrays to be concatenated
        data_to_concat = []
        if gfs_data is not None:
            data_to_concat.append(gfs_data)

        # Add the requested custom arrays to the list
        for var_name in requested_custom_vars:
            # Expand dims to match the structure of the fetched data
            custom_da = self.custom_arrays[var_name].expand_dims({"time": time})
            data_to_concat.append(custom_da)

        # Concatenate all data arrays along the 'variable' dimension
        if len(data_to_concat) > 1:
            return xr.concat(data_to_concat, dim="variable")
        elif len(data_to_concat) == 1:
            return data_to_concat[0]
        else:
            # Should not happen if 'variable' is never empty
            return xr.DataArray()


# Compare GFS init with and without surface geopotential and land mask
init_time = pd.Timestamp("20240501")

model = GraphCastOperational.load_model(GraphCastOperational.load_default_package())
ds = GFS()
# cache=False to avoid AttributeError: type object 'WholeFileCacheFileSystem'
# has no attribute '_cat_file'. Did you mean: 'cat_file'?
# dummy time list for required positional argument 'time'
# squeeze 'time' to avoid ValueError: Dimension time already exists.
z = SurfaceGeoPotential(cache=False)([None]).squeeze()
lsm = LandSeaMask(cache=False)([None]).squeeze()

# --- Instantiate Custom Data Source ---
custom_data = {"z": z, "lsm": lsm}
ds_filled = GFSWithSfcFields(custom_arrays=custom_data)

In [6]:
ds([init_time], model.input_coords()["variable"])

2025-11-13 13:13:31.784 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id tp06 not found in GFS lexicon, good luck
2025-11-13 13:13:31.784 | WARNING  | earth2studio.data.gfs:_create_tasks:324 - Variable tp06 not found in index file for time 2024-05-01 00:00:00 at 0:00:00, values will be unset
2025-11-13 13:13:31.786 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id z not found in GFS lexicon, good luck
2025-11-13 13:13:31.786 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id lsm not found in GFS lexicon, good luck
2025-11-13 13:13:31.786 | WARNING  | earth2studio.data.gfs:_create_tasks:324 - Variable lsm not found in index file for time 2024-05-01 00:00:00 at 0:00:00, values will be unset


Fetching GFS data:   0%|          | 0/83 [00:00<?, ?it/s]

2025-11-13 13:13:31.789 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 313058880-919336


Fetching GFS data:   0%|          | 0/83 [00:01<?, ?it/s]

2025-11-13 13:13:33.138 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 340138516-855908
2025-11-13 13:13:33.222 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 346962959-956469
2025-11-13 13:13:33.306 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 363874761-867028


...

Fetching GFS data:   0%|          | 0/83 [00:08<?, ?it/s]

2025-11-13 13:13:39.691 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 263369609-742260
2025-11-13 13:13:39.768 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 291198272-917359
2025-11-13 13:13:39.845 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 284818519-747033


Fetching GFS data: 100%|██████████| 83/83 [00:08<00:00, 10.16it/s]


2025-11-13 13:13:39.913 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 306502725-771192


<xarray.DataArray (time: 1, variable: 85, lat: 721, lon: 1440)> Size: 706MB
array([[[[ 2.60285400e+02,  2.60285400e+02,  2.60285400e+02, ...,
           2.60285400e+02,  2.60285400e+02,  2.60285400e+02],
         [ 2.60685394e+02,  2.60685394e+02,  2.60685394e+02, ...,
           2.60685394e+02,  2.60685394e+02,  2.60685394e+02],
         [ 2.60885406e+02,  2.60885406e+02,  2.60885406e+02, ...,
           2.60885406e+02,  2.60885406e+02,  2.60885406e+02],
         ...,
         [ 2.40385406e+02,  2.40385406e+02,  2.40385406e+02, ...,
           2.40385406e+02,  2.40385406e+02,  2.40385406e+02],
         [ 2.40185394e+02,  2.40185394e+02,  2.40185394e+02, ...,
           2.40185394e+02,  2.40185394e+02,  2.40185394e+02],
         [ 2.40085403e+02,  2.40085403e+02,  2.40085403e+02, ...,
           2.40085403e+02,  2.40085403e+02,  2.40085403e+02]],

        [[ 1.03238914e+05,  1.03238914e+05,  1.03238914e+05, ...,
           1.03238914e+05,  1.03238914e+05,  1.03238914e+05],
         [ 1.03228516e+05,  1.03228516e+05,  1.03228914e+05, ...,
           1.03227711e+05,  1.03228109e+05,  1.03228109e+05],
         [ 1.03217711e+05,  1.03218109e+05,  1.03218516e+05, ...,
           1.03216109e+05,  1.03216516e+05,  1.03216914e+05],
...
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]]],
      shape=(1, 85, 721, 1440))
Coordinates:
  * time       (time) datetime64[ns] 8B 2024-05-01
    lead_time  timedelta64[ns] 8B 00:00:00
  * variable   (variable) <U5 2kB 't2m' 'msl' 'u10m' ... 'q1000' 'z' 'lsm'
  * lat        (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon        (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8

In [7]:
# --- Run Forecast ---
nsteps = 8
ofile = SCRATCH / "tmp/GFS.nc"
if os.path.exists(ofile):
    os.remove(ofile)
io = NetCDF4Backend(ofile, backend_kwargs={"mode": "w"})
deterministic([init_time], nsteps, model, ds, io)
io.close()
ofile = SCRATCH / "tmp/GFSWithSfcFields.nc"
if os.path.exists(ofile):
    os.remove(ofile)
io = NetCDF4Backend(ofile, backend_kwargs={"mode": "w"})
deterministic([init_time], nsteps, model, ds_filled, io)
io.close()

print("Forecast run complete.")

2025-11-13 13:14:41.991 | INFO     | earth2studio.run:deterministic:75 - Running simple workflow!
2025-11-13 13:14:42.004 | INFO     | earth2studio.run:deterministic:82 - Inference device: cuda
2025-11-13 13:14:42.048 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id tp06 not found in GFS lexicon, good luck
2025-11-13 13:14:42.048 | WARNING  | earth2studio.data.gfs:_create_tasks:324 - Variable tp06 not found in index file for time 2024-04-30 18:00:00 at 0:00:00, values will be unset
2025-11-13 13:14:42.050 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id z not found in GFS lexicon, good luck
2025-11-13 13:14:42.050 | WARNING  | earth2studio.data.gfs:_create_tasks:306 - variable id lsm not found in GFS lexicon, good luck
2025-11-13 13:14:42.050 | WARNING  | earth2studio.data.gfs:_create_tasks:324 - Variable lsm not found in index file for time 2024-04-30 18:00:00 at 0:00:00, values will be unset


Fetching GFS data:   0%|          | 0/83 [00:00<?, ?it/s]

2025-11-13 13:14:42.053 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 370505196-1159605
2025-11-13 13:14:42.136 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 163871432-862889


Fetching GFS data:   0%|          | 0/83 [00:00<?, ?it/s]

2025-11-13 13:14:42.209 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240430/18/atmos/gfs.t18z.pgrb2.0p25.f000 403318105-932801


...

Fetching GFS data:   0%|          | 0/83 [00:03<?, ?it/s]

2025-11-13 13:14:51.728 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 267700128-806123
2025-11-13 13:14:51.768 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 370718395-966063


Fetching GFS data: 100%|██████████| 83/83 [00:03<00:00, 24.87it/s]


2025-11-13 13:14:51.808 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 289201927-1073631
2025-11-13 13:14:51.848 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 248219609-586527
2025-11-13 13:14:51.887 | DEBUG    | earth2studio.data.gfs:fetch_array:380 - Fetching GFS grib file: noaa-gfs-bdp-pds/gfs.20240501/00/atmos/gfs.t00z.pgrb2.0p25.f000 216915645-611921


OutOfMemoryError: CUDA out of memory. Tried to allocate 674.00 MiB. GPU 0 has a total capacity of 31.73 GiB of which 605.69 MiB is free. Process 60130 has 24.11 GiB memory in use. Including non-PyTorch memory, this process has 7.03 GiB memory in use. Of the allocated memory 0 bytes is allocated by PyTorch, and 0 bytes is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
ds(init_time, model.input_coords()["variable"])

In [ ]:
ds.sel(variable="lsm")

In [ ]:
ifile = SCRATCH / f"ai-models/output/graphcast/{init_time:%Y%m%d%H}/gep01.nc"
print(ifile)
ds = xr.open_dataset(ifile)
ds.z500.squeeze().max(dim=["lat", "lon"]) < 1e30

In [ ]:
ds = xr.open_dataset(SCRATCH / f"ai-models/output/graphcast/{init_time:%Y%m%d%H}/gep01.nc")
ds.z100.squeeze().max(dim=["lat", "lon"])

In [ ]:
ds = xr.open_zarr(SCRATCH / f"ai-models/output/graphcast/{init_time:%Y%m%d%H}/gep23")
ds

In [ ]:
ds.squeeze().max(dim=["lat", "lon"]).load()